# Forecast Verification vs ERA5 (Europe, 2m Temperature)

Compare **IFS** and **AIFS** forecasts against ERA5 reanalysis ground truth for `t2m` over Europe.

In [92]:
from pathlib import Path
import numpy as np
import xarray as xr

ifs_path = Path("data/IFS_forecast_europe.nc")
aifs_path = Path("data/AIFS_forecast_europe.nc")

assert ifs_path.exists(), f"Missing {ifs_path}"
assert aifs_path.exists(), f"Missing {aifs_path}"
print("Forecast files found")

Forecast files found


## Load & Inspect Forecast Files
Both files are initialized at 2026-01-01 00:00 UTC, covering Europe at 1° resolution.

In [93]:
ds_ifs = xr.open_dataset(ifs_path)
ds_aifs = xr.open_dataset(aifs_path)
print(f"IFS - {dict(ds_ifs.sizes)}")
print(f"AIFS - {dict(ds_aifs.sizes)}")

IFS - {'time': 145, 'latitude': 41, 'longitude': 73}
AIFS - {'time': 61, 'latitude': 41, 'longitude': 73}


In [ ]:
def inspect_forecast(ds, name):
    """Print key properties of a forecast dataset for sanity checking."""
    t2m = ds["t2m"]
    lat = ds["latitude"].values
    lon = ds["longitude"].values
    times = ds["time"].values
    lat_dir = "N to S" if lat[0] > lat[-1] else "S to N"
    lon_dir = "W to E" if lon[0] < lon[-1] else "E to W"

    print(f"{name}:")
    print(f"  Grid:       {len(lat)} lat x {len(lon)} lon (1 deg)")
    print(f"  Lat:        {lat[0]:.0f} to {lat[-1]:.0f} ({lat_dir})")
    print(f"  Lon:        {lon[0]:.0f} to {lon[-1]:.0f} ({lon_dir})")
    print(f"  Timesteps:  {len(times)}")
    print(f"  Time range: {str(times[0])[:16]} to {str(times[-1])[:16]}")
    print(f"  Temp range: {float(t2m.min()):.1f} K to {float(t2m.max()):.1f} K")
    print(f"  NaN count:  {int(t2m.isnull().sum())}")
    print()

inspect_forecast(ds_ifs, "IFS")
inspect_forecast(ds_aifs, "AIFS")

# Grid alignment check
lat_match = np.array_equal(ds_ifs["latitude"].values, ds_aifs["latitude"].values)
lon_match = np.array_equal(ds_ifs["longitude"].values, ds_aifs["longitude"].values)
assert lat_match and lon_match, "Grid mismatch between IFS and AIFS"
print("IFS and AIFS grids match, no regridding needed")

## Find Overlapping Forecast Times

In [95]:
overlap_times = np.intersect1d(ds_ifs["time"].values, ds_aifs["time"].values)

print(f"IFS timesteps:  {len(ds_ifs['time'])}")
print(f"AIFS timesteps: {len(ds_aifs['time'])}")
print(f"Overlap:        {len(overlap_times)} timesteps")
print(f"First: {overlap_times[0]}")
print(f"Last:  {overlap_times[-1]}")

# Verify uniform 6-hour spacing
diffs = np.diff(overlap_times).astype("timedelta64[h]").astype(int)
print(f"Time spacing: every {diffs[0]}h (uniform: {np.all(diffs == diffs[0])})")

# Lead time in hours since forecast init
init_time = overlap_times[0]  # 2026-01-01 00:00 UTC
end_time = overlap_times[-1]

lead_hours = ((overlap_times - init_time) / np.timedelta64(1, "h")).astype(int)
print(f"Lead time range: {lead_hours[0]}h to {lead_hours[-1]}h ({lead_hours[-1]/24:.0f} days)")

IFS timesteps:  145
AIFS timesteps: 61
Overlap:        61 timesteps
First: 2026-01-01T00:00:00.000000000
Last:  2026-01-16T00:00:00.000000000
Time spacing: every 6h (uniform: True)
Lead time range: 0h to 360h (15 days)
